# Solution

In [52]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm
from scipy.sparse import csr_matrix
from typing import Optional, Union
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from scipy.sparse import diags

In [54]:
train_df = pd.read_csv("training.csv")
test_df = pd.read_csv("testset.csv")

In [55]:
train_df.head()

,userid,movieid,rating,timestamp
0,158862,1600,4.0,978307288
1,49693,9102,4.0,985466347
2,65237,2408,3.0,978308050
3,82765,2590,2.0,985139361
4,120946,285,2.0,978308465


In [56]:
train_df.describe()

,userid,movieid,rating,timestamp
count,1.178277e+07,1.178277e+07,1.178277e+07,1.178277e+07
mean,1.419319e+05,1.919155e+03,3.502378e+00,1.158040e+09
std,8.179844e+04,2.822795e+03,1.059831e+00,1.034930e+08
min,0.000000e+00,0.000000e+00,5.000000e-01,9.783073e+08
25%,7.142000e+04,3.700000e+02,3.000000e+00,1.077814e+09
50%,1.421970e+05,8.910000e+02,3.500000e+00,1.144422e+09
75%,2.130310e+05,2.113000e+03,4.000000e+00,1.236016e+09
max,2.832270e+05,5.385900e+04,5.000000e+00,1.380320e+09


In [57]:
train_users = set(train_df['userid'].unique())
test_users = set(test_df['userid'].unique())

users_in_train = test_users.intersection(train_users)
users_not_in_train = test_users.difference(train_users)

print(f"Total users in test set: {len(test_users)}")
print(f"Users present in training set: {len(users_in_train)}")
print(f"Users NOT present in training set: {len(users_not_in_train)}")

Total users in test set: 2963
Users present in training set: 0
Users NOT present in training set: 2963


In [58]:
top_popular_movies_count_train = (
    train_df.groupby('movieid')
    .size()
    .sort_values(ascending=False)
    .head(7250)
    .index.tolist()
)

top_popular_movies_rating_train = (
    train_df.groupby('movieid')['rating']
    .mean()
    .sort_values(ascending=False)
    .head(7250)
    .index.tolist()
)

movie_counts = train_df['movieid'].value_counts() 
total_interactions = len(train_df) 

popular_count = sum(movie_counts.get(movie, 0) for movie in top_popular_movies_count_train)
popular_rating = sum(movie_counts.get(movie, 0) for movie in top_popular_movies_rating_train)

percentage_count = (popular_count / total_interactions) * 100
percentage_rating = (popular_rating / total_interactions) * 100

print(f"Percentage of users in train that have seen top 4500 most watched films: {percentage_count:.2f}%")
print(f"Percentage of users in train that have seen top 4500 rated films: {percentage_rating:.2f}%")

Percentage of users in train that have seen top 4500 most watched films: 98.25%
Percentage of users in train that have seen top 4500 rated films: 61.13%


As wee see almost 98% of all movies can be explained by 7250 most watched

### Some useful functions

In [66]:
def timepoint_split(data, time_split_q=0.95):
    """
    Split data into training, testset, and holdout datasets based on a timepoint split
    and according to the `warm-start` evaluation strategy.

    Parameters
    ----------
    data : pd.DataFrame
        The input dataset containing columns `userid`, `movieid`, and `timestamp`.
    time_split_q : float, optional
        The quantile value used to split the dataset based on the `timestamp` column.
        Default is 0.95.

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]
        A tuple of three pandas DataFrames: training, testset, and holdout.
        `training` is a subset of `data` used for training the recommender system.
        `testset` is a subset of `data` used for generating recommendations for the test users.
        `holdout` is a subset excluded from `testset` containing only the most recent interactions for each test user.

    Notes
    -----
    The function splits the input `data` into three subsets: `training`, `testset`, and `holdout`.
    The split is performed based on the `timestamp` column of `data`, using `time_split_q` as the quantile value.
    The `holdout` dataset contains only the immediate interactions following the fixed timepoint for each test user from the `testset`.
    The set of users in `training` is disjoint with the set of users in the `testset`, which implements the `warm-start` scenario.
    """    
    timepoint = data.timestamp.quantile(q=time_split_q, interpolation='nearest')
    test_ = data.query('timestamp >= @timepoint')
    rest_ = data.drop(test_.index)
    holdout_ = (
        test_
        .sort_values('timestamp')
        .drop_duplicates(subset=['userid'], keep='first')
    )
    # the holdout dataframe contains interactions closest to certain timepoint from the right,
    # i.e., the corresponding items are the first in each test user profile after this timepoint
    training = rest_.query('userid not in @holdout_.userid')
    train_items = training.movieid.unique()
    testset_ = rest_.query('userid in @holdout_.userid and movieid in @train_items')
    test_users = testset_.userid.unique()
    holdout = holdout_.query(
        # if user is not in `test_users` then no evluation is possible,
        # if item is not in `train_items` it's cold start -> must be excluded
        'userid in @test_users and movieid in @train_items'
    ).sort_values('userid')
    testset = testset_.query(
        # make sure testset and holdout contain the same set of users
        'userid in @holdout.userid'
    ).sort_values('userid')
    return training, testset, holdout

def leave_last_out(data, userid='userid', timeid='timestamp'):
    data_sorted = data.sort_values('timestamp')
    holdout = data_sorted.drop_duplicates(
        subset=['userid'], keep='last'
    ) # split the last item from each user's history
    remaining = data.drop(holdout.index) # store the remaining data - will be our training
    return remaining, holdout

def transform_indices(data: pd.DataFrame, users: str, items:str, inplace: bool=False):
    '''
    Reindex columns that correspond to users and items.
    New index is contiguous starting from 0.

    Parameters
    ----------
    data : pandas.DataFrame
        The input data to be reindexed.
    users : str
        The name of the column in `data` that contains user IDs.
    items : str
        The name of the column in `data` that contains item IDs.
    inplace : bool
        whether the data should be modified inplace, `False` by default.

    Returns
    -------
    pandas.DataFrame, dict
        The reindexed data and a dictionary with mapping between original IDs and the new numeric IDs.
        The keys of the dictionary are 'users' and 'items'.
        The values of the dictionary are pandas Index objects.

    Examples
    --------
    >>> data = pd.DataFrame({'customers': ['A', 'B', 'C'], 'products': ['X', 'Y', 'Z'], 'rating': [1, 2, 3]})
    >>> data_reindexed, data_index = transform_indices(data, 'customers', 'products')
    >>> data_reindexed
       users  items  rating
    0      0      0       1
    1      1      1       2
    2      2      2       3
    >>> data_index
    {
        'users': Index(['A', 'B', 'C'], dtype='object', name='customers'),
        'items': Index(['X', 'Y', 'Z'], dtype='object', name='products')
    }
    '''
    data_index = {}
    data_codes = {}
    for entity, field in zip(['users', 'items'], [users, items]):
        new_index, data_index[entity] = to_numeric_id(data, field)
        if inplace:
            data.loc[:, field] = new_index
        else:
            data_codes[field] = new_index

    if data_codes:
        data = data.assign(**data_codes) # makes a copy of data
    return data, data_index


def to_numeric_id(data: pd.DataFrame, field: str):
    """
    This function takes in two arguments, data and field. It converts the data field
    into categorical values and creates a new contiguous index. It then creates an
    idx_map which is a renamed version of the field argument. Finally, it returns the
    idx and idx_map variables.
    """
    idx_data = data[field].astype("category")
    idx = idx_data.cat.codes
    idx_map = idx_data.cat.categories.rename(field)
    return idx, idx_map


def reindex_data(
        data: pd.DataFrame,
        data_index: dict,
        entities: Optional[Union[str, list[str]]] = None,
        filter_invalid: bool = True,
        inplace: bool = False
    ):
    '''
    Reindex provided data with the specified index mapping.
    By default, will take the name of the fields to reindex from `data_index`.
    It is also possible to specify which field to reindex by providing `entities`.
    '''
    if entities is None:
        entities = data_index.keys()
    if isinstance(entities, str): # handle single entity provided as a string
        entities = [entities]

    data_codes = {}
    for entity in entities:
        entity_index = data_index[entity]
        field = entity_index.name # extract the field name
        new_index = entity_index.get_indexer(data[field])
        if inplace:
            data.loc[:, field] = new_index # assign new values inplace
        else:
            data_codes[field] = new_index # store new values

    if data_codes:
        data = data.assign(**data_codes) # assign new values by making a copy

    if filter_invalid: # discard unrecognized entity index
        valid_values = [f'{data_index[entity].name}>=0' for entity in entities]
        data = data.query(' and '.join(valid_values))
    return data


def generate_interactions_matrix(
        data: pd.DataFrame,
        data_description: dict,
        rebase_users: bool=False
    ):
    '''
    Converts a pandas dataframe with user-item interactions into a sparse matrix representation.
    Allows reindexing user ids, which help ensure data consistency at the scoring stage
    (assumes user ids are sorted in the scoring array).

    Args:
        data (pandas.DataFrame): The input dataframe containing the user-item interactions.
        data_description (dict): A dictionary containing the data description with the following keys:
            - 'n_users' (int): The total number of unique users in the data.
            - 'n_items' (int): The total number of unique items in the data.
            - 'users' (str): The name of the column in the dataframe containing the user ids.
            - 'items' (str): The name of the column in the dataframe containing the item ids.
            - 'feedback' (str): The name of the column in the dataframe containing the user-item interaction feedback.
        rebase_users (bool, optional): Whether to reindex the user ids to make contiguous index starting from 0. Defaults to False.

    Returns:
        scipy.sparse.csr_matrix: A sparse matrix of shape (n_users, n_items) containing the user-item interactions.
    '''

    n_users = data_description['n_users']
    n_items = data_description['n_items']
    # get indices of observed data
    user_idx = data[data_description['users']].values
    if rebase_users: # handle non-contiguous index of test users
        # This ensures that all user ids are contiguous and start from 0,
        # which helps ensure data consistency at the scoring stage.
        user_idx, user_index = pd.factorize(user_idx, sort=True)
        n_users = len(user_index)
    item_idx = data[data_description['items']].values
    feedback = data[data_description['feedback']].values
    # construct rating matrix
    return csr_matrix((feedback, (user_idx, item_idx)), shape=(n_users, n_items))


def verify_time_split(
        before: pd.DataFrame,
        after: pd.DataFrame,
        target_field: str='userid',
        timeid: str='timestamp'
    ):
    '''
    Check that items from `after` dataframe have later timestamps than
    any corresponding item from the `before` dataframe. Compare w.r.t target_field.
    Usage example: assert that for any user, the holdout items are the most recent ones.
    '''
    before_ts = before.groupby(target_field)[timeid].max()
    after_ts = after.groupby(target_field)[timeid].min()
    assert (
        before_ts
        .reindex(after_ts.index)
        .combine(after_ts, lambda x, y: True if x!=x else x <= y)
    ).all()

def downvote_seen_items(scores: np.ndarray, data: pd.DataFrame, data_description: dict) -> None:
    """
    Downvote relevance scores for seen items.

    Parameters
    ----------
    scores : np.ndarray
        A dense numpy array of scores.
    data : pd.DataFrame
        A pandas DataFrame containing user-item interaction data.
    data_description : dict
        A dictionary containing metadata about the data, including 'items' and 'users' keys.

    Returns
    -------
    None
        The function modifies the input scores array in-place.

    Notes
    -----
    This function assumes that the input scores array is sorted by user index.
    It also assumes that the `users` and `items` keys in the `data_description` dictionary
    correspond to the column names in the `data` DataFrame.
    """
    assert isinstance(scores, np.ndarray), 'Scores must be a dense numpy array!'
    itemid = data_description['items']
    userid = data_description['users']
    # get indices of observed data, corresponding to scores array
    # we need to provide correct mapping of rows in scores array into
    # the corresponding user index (which is assumed to be sorted)
    row_idx, test_users = pd.factorize(data[userid], sort=True)
    assert len(test_users) == scores.shape[0]
    col_idx = data[itemid].values
    # downvote scores at the corresponding positions
    scores[row_idx, col_idx] = scores.min() - 1


def topn_recommendations(scores: np.ndarray, topn: int=10) -> np.ndarray:
    """
    Generate top-N recommendations based on the input scores.

    Parameters
    ----------
    scores : np.ndarray
        A dense numpy array of scores, where each row corresponds to a user and each column to an item.
    topn : int, optional
        The number of top recommendations to generate for each user. Defaults to 10.

    Returns
    -------
    np.ndarray
        A numpy array of shape (n_users, topn) containing the indices of the top-N recommended items for each user.
    """
    recommendations = np.apply_along_axis(topidx, 1, scores, topn)
    return recommendations


def topidx(a: np.ndarray, topn: int) -> np.ndarray:
    """
    Returns the indices of the top-N elements in the input array.
    The calculation is based on the argpartition method, which is more efficient for large arrays.

    Parameters
    ----------
    a : np.ndarray
        The input array from which to select the top-N elements.
    topn : int
        The number of top elements to select.

    Returns
    -------
    np.ndarray
        An array of indices corresponding to the top-N elements in the input array.
    """
    parted = np.argpartition(a, -topn)[-topn:]
    return parted[np.argsort(-a[parted])]


def model_evaluate(
        recommended_items: np.ndarray,
        holdout: pd.DataFrame,
        holdout_description: dict,
        topn: int = 20
    ) -> tuple:
    """
    Evaluates the performance of a recommender model by comparing the recommended items against the holdout.

    Parameters
    ----------
    recommended_items : np.ndarray
        A numpy array of shape (n_users, topn) containing the indices of the top-N recommended items for each user.
    holdout : pd.DataFrame
        A pandas DataFrame containing the holdout data for evaluation.
    holdout_description : dict
        A dictionary containing the description of the holdout data, including the column names for items.
    topn : int, optional
        The number of top recommendations to consider for evaluation. Defaults to 10.

    Returns
    -------
        NDCG@k
    """
    itemid = holdout_description['items']
    holdout_items = holdout[itemid].values
    assert recommended_items.shape[0] == len(holdout_items)

    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    n_test_users = recommended_items.shape[0]
    hit_rank = np.where(hits_mask)[1] + 1.0
    ndcg_pu = 1.0 / np.log2(hit_rank + 1)
    ndcg = np.sum(ndcg_pu) / n_test_users
    return ndcg


def calculate_metrics(recommendations_df, test_df, user_col='userid', item_col='movieid', k=20):
    """
    Calculates HR@k, MRR@k, Cov@k, and NDCG@k for the recommendations.

    Parameters:
    -----------
    recommendations_df : pd.DataFrame
        The recommendations DataFrame with columns 'userid' and 'movieid'.
    test_df : pd.DataFrame
        The test set DataFrame with columns 'userid' and 'movieid'.
    user_col : str
        The name of the user column.
    item_col : str
        The name of the item column.
    k : int
        The number of recommendations to consider (e.g., 20 for HR@20, MRR@20, Cov@20, NDCG@20).

    Returns:
    --------
    Tuple[float, float, float, float]
        The HR@k, MRR@k, Cov@k, and NDCG@k scores.
    """
    ground_truth = test_df.groupby(user_col)[item_col].apply(set).to_dict()
    
    hits = 0
    reciprocal_ranks = []
    recommended_items = set()
    total_ndcg = 0.0
    
    for user, recommendations in recommendations_df.groupby(user_col)[item_col]:
        true_items = ground_truth.get(user, set())

        recommended_items_list = recommendations.tolist()[:k]
        
        recommended_items.update(recommended_items_list)
        
        if any(item in true_items for item in recommended_items_list):
            hits += 1
        
        for rank, item in enumerate(recommended_items_list, start=1):
            if item in true_items:
                reciprocal_ranks.append(1.0 / rank)
                break
        
        dcg = 0.0
        for i, item in enumerate(recommended_items_list):
            if item in true_items:
                dcg += 1.0 / np.log2(i + 2) 
        
        idcg = 0.0
        for i in range(min(len(true_items), k)):
            idcg += 1.0 / np.log2(i + 2)
        
        ndcg = dcg / idcg if idcg > 0 else 0.0
        total_ndcg += ndcg
    
    # HR@k
    hr = hits / len(ground_truth)
    
    # MRR@k
    mrr = np.mean(reciprocal_ranks) if reciprocal_ranks else 0.0
    
    # Cov@k
    unique_items_in_test = set(test_df[item_col].unique())
    cov = len(recommended_items) / len(unique_items_in_test)

    # NDCG@k
    ndcg = total_ndcg / len(ground_truth)
    
    return hr, mrr, cov, ndcg

def recommendations_to_df(recommendations, test_users_ids, reverse_movie_id_map):
    user_ids = []
    movie_ids = []
    
    for user_idx, items in enumerate(recommendations):
        user_id = test_users_ids[user_idx]
        
        remapped_items = [reverse_movie_id_map[item_idx] for item_idx in items]
        
        user_ids.extend([user_id] * len(remapped_items))
        movie_ids.extend(remapped_items)
    
    return pd.DataFrame({'userid': user_ids, 'movieid': movie_ids})

Let's select only 4500 most watched films as they describe 95% of all interactions

In [69]:
top_movies = train_df['movieid'].value_counts().nlargest(7250).index

train_filtered = train_df[train_df['movieid'].isin(top_movies)]

data_description = dict(
    users='userid',
    items='movieid',
    feedback='rating',
    n_users=train_filtered.userid.nunique(),
    n_items=train_filtered.movieid.nunique()
)

In [70]:
training, testset, holdout = timepoint_split(train_filtered, time_split_q=0.95)

In [72]:
training, data_index = transform_indices(training, 'userid', 'movieid')
testset = reindex_data(testset, data_index, entities='items')
holdout = reindex_data(holdout, data_index, entities='items')

For recommendation generation we will use Trunctated SVD with decay over timestamp and scaling within users

In [128]:
def scale_matrix(matrix, alpha=0.5):
    """Scales the interaction matrix using the alpha parameter."""
    item_interactions = matrix.getnnz(axis=0)
    scaling_factors = np.power(item_interactions, (alpha - 1) / 2)
    D = diags(scaling_factors)
    return matrix.dot(D)

def normalize_timestamps(timestamps, min_timestamp, max_timestamp):
    """Normalizes timestamps to a range of [0, 1]."""
    return (timestamps - min_timestamp) / (max_timestamp - min_timestamp)

def compute_decay_weights(normalized_timestamps, decay_rate):
    """Computes decay weights for interactions based on normalized timestamps."""
    return np.power(decay_rate, 1 - normalized_timestamps)

def apply_decay_to_matrix(data, decay_rate, timestamp_col='timestamp', feedback_col='rating'):
    """Applies decay weights to the interaction matrix based on timestamps."""
    min_timestamp = data[timestamp_col].min()
    max_timestamp = data[timestamp_col].max()
    normalized_timestamps = normalize_timestamps(data[timestamp_col], min_timestamp, max_timestamp)
    decay_weights = compute_decay_weights(normalized_timestamps, decay_rate)
    decayed_feedback = data[feedback_col] * decay_weights
    user_idx = data['userid'].values
    item_idx = data['movieid'].values
    return csr_matrix((decayed_feedback, (user_idx, item_idx)), shape=(data['userid'].nunique(), data['movieid'].nunique()))

# ------------------------
# 2️⃣ Training SVD Model
# ------------------------
def build_svd_model(training, rank, alpha, decay_rate):
    """Trains SVD model only on training data (no test users)."""
    decayed_matrix = apply_decay_to_matrix(training, decay_rate)
    scaled_matrix = scale_matrix(decayed_matrix, alpha)
    U, sigma, Vt = svds(scaled_matrix, k=rank)
    return U, np.diag(sigma), Vt

# ------------------------
# 3️⃣ Folding-in Test Users
# ------------------------
def fold_in_users(U, Vt, testset, data_description, n_epochs=10, lr=0.01, reg=0.1):
    """Infers test user embeddings using pre-trained item factors."""

    user_ids = testset[data_description['users']].values
    item_idx = testset[data_description['items']].values
    ratings = testset[data_description['feedback']].values

    unique_users = np.unique(user_ids)
    user_map = {uid: idx for idx, uid in enumerate(unique_users)}  # Map user ID → local index

    n_users_test = len(unique_users)
    rank = Vt.shape[0]

    P_test = np.zeros((n_users_test, rank))  # Correct shape

    for epoch in range(n_epochs):
        for i in range(len(user_ids)):
            user = user_map[user_ids[i]]  # Convert user ID to test index
            item = item_idx[i]
            rating = ratings[i]

            qi = Vt.T[item]  # Item factor (rank x 1)
            pu = P_test[user]  # User factor (1 x rank)

            err = rating - np.dot(pu, qi)  # Correct matrix multiplication
            P_test[user] += lr * (err * qi - reg * pu)  # Update step

    return P_test


# ------------------------
# 4️⃣ Compute Scores
# ------------------------
def svd_scoring(P_test, Vt, topn=20):
    """Generates predicted scores for test users."""
    scores = P_test @ Vt
    return scores

# ------------------------
# 5️⃣ Generate Recommendations
# ------------------------
def topn_recommendations(scores, topn=20):
    """Generates top-N recommendations."""
    return np.argsort(-scores, axis=1)[:, :topn]

# ------------------------
# 6️⃣ Evaluate Recommendations
# ------------------------
import numpy as np

def dcg_at_k(ranked_relevances, k):
    """Computes Discounted Cumulative Gain (DCG) at k."""
    ranked_relevances = np.asarray(ranked_relevances)[:k]
    return np.sum(ranked_relevances / np.log2(np.arange(2, k + 2)))

def ndcg_at_k(recommended_items, holdout_items, k=20):
    """Computes Normalized DCG (NDCG) at k."""
    dcg = 0
    idcg = dcg_at_k([1] * min(len(holdout_items), k), k)  # Ideal DCG

    for i, user_recs in enumerate(recommended_items):
        user_holdout = {holdout_items[i]}  # ✅ Convert single item to a set
        relevance = [1 if item in user_holdout else 0 for item in user_recs]  # Binary relevance
        dcg += dcg_at_k(relevance, k)

    return (dcg / len(recommended_items)) / idcg if idcg > 0 else 0


def model_evaluate(recommended_items, holdout, holdout_description, topn=20):
    """Evaluates the model using HR@10, MRR@10, Coverage, and NDCG@20."""
    itemid = holdout_description['items']
    holdout_items = holdout[itemid].values
    assert recommended_items.shape[0] == len(holdout_items)

    # Hit Rate (HR@10)
    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    hr = np.mean(hits_mask.any(axis=1))

    # Mean Reciprocal Rank (MRR@10)
    hit_rank = np.where(hits_mask)[1] + 1.0
    mrr = np.sum(1 / hit_rank) / recommended_items.shape[0]

    # Coverage
    n_items = holdout_description['n_items']
    coverage = np.unique(recommended_items).size / n_items

    # NDCG@20
    ndcg = ndcg_at_k(recommended_items, holdout_items, k=20)

    return ndcg, hr, mrr, coverage


# ------------------------
# 7️⃣ Cross-Validation for Best Rank & Alpha
# ------------------------
def cross_validate_svd(training, testset, holdout, data_description, rank_values, alpha_values, decay_rates):
    """Performs cross-validation to find the best rank & alpha using SVD."""
    results = {}

    for decay_rate in decay_rates:
        for alpha in alpha_values:
            max_rank = max(rank_values)
            _, _, Vt = build_svd_model(training, max_rank, alpha, decay_rate)
    
            for rank in tqdm(rank_values, desc='GridSearch for the best alpha & rank'):
                Vt_rank = Vt[:rank, :]
                P_test = fold_in_users(_, Vt_rank, testset, data_description)
    
                scores = svd_scoring(P_test, Vt_rank, topn=20)
                recommendations = topn_recommendations(scores, topn=20)
    
                ndcg, ndcghr, mrr, coverage = model_evaluate(recommendations, holdout, data_description, topn=20)
                results[(rank, alpha, decay_rate)] = (hr, mrr, coverage, ndcg)

    best_rank, best_alpha, best_decay_rate = max(results, key=lambda x: results[x][0])  # Select based on HR@10
    best_metrics = results[(best_rank, best_alpha, best_decay_rate)]
    
    return best_rank, best_alpha, best_decay_rate, best_metrics, results

# ------------------------
# 8️⃣ Running Everything
# ------------------------
data_description = {
    "users": "userid",
    "items": "movieid",
    "feedback": "rating",
    "n_users": training.userid.nunique(),
    "n_items": training.movieid.nunique()
}

# Set hyperparameters
decay_rates = [0.93, 0.95, 0.97, 0.99, 1.01, 1.03]
alpha_values = [0.35, 0.45, 0.55]
rank_values = np.arange(100, 501, 50).tolist()

# Run cross-validation
best_rank, best_alpha, best_decay_rate, best_metrics, results = cross_validate_svd(
    training, testset, holdout, data_description, rank_values, alpha_values, decay_rates
)

print(f"✅ Best Rank: {best_rank}, Best Alpha: {best_alpha}, Best Decay: {best_decay_rate}, ndcg@20: {best_metrics[0]:.3f}")


GridSearch for the best alpha & rank:   0%|               | 0/9 [00:43<?, ?it/s]


AssertionError: 

In [239]:
def scale_matrix(matrix, alpha=0.5):
    """
    Scales the interaction matrix using the alpha parameter
    """
    item_interactions = matrix.getnnz(axis=0) 
    
    scaling_factors = np.power(item_interactions, (alpha - 1) / 2)
    
    D = diags(scaling_factors)
    
    scaled_matrix = matrix.dot(D)
    
    return scaled_matrix

def build_svd_model(matrix, rank, alpha, decay_rate):
    """
    Builds the SVD model for the given rank and alpha
    """
    decayed_matrix = apply_decay_to_matrix(matrix, decay_rate)
    scaled_matrix = scale_matrix(decayed_matrix, alpha)
    U, sigma, Vt = svds(scaled_matrix, k=rank)
    sigma = np.diag(sigma)
    return U, sigma, Vt

def cross_validate_svd(training, data_description, holdout, rank_values, alpha_values, decay_rate):
    """
    Performs cross-validation to find the best rank and alpha using SVD
    Builds the SVD model once for the maximum rank and shrinks it for smaller ranks
    """
    results = {}
    
    #train_matrix = generate_interactions_matrix(training, data_description)
    
    for alpha in alpha_values:
        
        max_rank = max(rank_values)
        *_, Vt = build_svd_model(training, max_rank, alpha, decay_rate)
        
        for rank in tqdm(rank_values, desc='GridSearching for the best alpha and rank'):
            Vt_rank = Vt[:rank, :]
            
            item_factors = Vt_rank.T
            
            scores = svd_scoring(item_factors, holdout, data_description)
            
            recommendations = topn_recommendations(scores, topn=20)
            
            ndcg = model_evaluate(recommendations, holdout, data_description, topn=20)
            
            results[(rank, alpha)] = ndcg
    
    best_rank, best_alpha = max(results, key=results.get)
    best_ndcg = results[(best_rank, best_alpha)]
    
    return best_rank, best_alpha, best_ndcg, results

def svd_scoring(item_factors, holdout, data_description):
    """
    Generates scores for the holdout set using the item factors
    """
    test_matrix = generate_interactions_matrix(holdout, data_description, rebase_users=True)
    print(test_matrix.shape, item_factors.shape)
    scores = test_matrix.dot(item_factors) @ item_factors.T
    return scores

def recommendations_to_df(recommendations, test_users_ids, reverse_movie_id_map):
    user_ids = []
    movie_ids = []
    
    for user_idx, items in enumerate(recommendations):
        user_id = test_users_ids[user_idx]
        
        remapped_items = [reverse_movie_id_map[item_idx] for item_idx in items]
        
        user_ids.extend([user_id] * len(remapped_items))
        movie_ids.extend(remapped_items)
    
    return pd.DataFrame({'userid': user_ids, 'movieid': movie_ids})


def normalize_timestamps(timestamps, min_timestamp, max_timestamp):
    """
    Normalizes timestamps to a range of [0, 1]
    """
    return (timestamps - min_timestamp) / (max_timestamp - min_timestamp)

def compute_decay_weights(normalized_timestamps, decay_rate):
    """
    Computes decay weights for interactions based on normalized timestamps
    """
    return np.power(decay_rate, 1 - normalized_timestamps)

def apply_decay_to_matrix(data, decay_rate, timestamp_col='timestamp', feedback_col='rating'):
    """
    Applies decay weights to the interaction matrix based on timestamps
    """
    min_timestamp = data[timestamp_col].min()
    max_timestamp = data[timestamp_col].max()
    normalized_timestamps = (data[timestamp_col] - min_timestamp) / (max_timestamp - min_timestamp) 
    
    decay_weights = compute_decay_weights(normalized_timestamps, decay_rate)
    
    decayed_feedback = data[feedback_col] * decay_weights
    
    user_idx = data['userid'].values
    item_idx = data['movieid'].values
    return csr_matrix((decayed_feedback, (user_idx, item_idx)), shape=(data['userid'].nunique(), data['movieid'].nunique()))

### Cross-validation process

In [18]:
# data_description = dict(
#     users='userid',
#     items='movieid',
#     feedback='rating',
#     n_users=train_filtered.userid.nunique(),
#     n_items=train_filtered.movieid.nunique()
# )

# decay_rate = 0.93
# alpha_values = [0.35]
# rank_values = np.arange(130, 301, 5).tolist()

# best_rank, best_alpha, best_ndcg, results = cross_validate_svd(
#     training, data_description, holdout, rank_values, alpha_values, decay_rate
# )

# print(f"Best rank: {best_rank} with NDCG@20: {best_ndcg}")

### Final model

In [171]:
top_movies = train_df['movieid'].value_counts().nlargest(7250).index

train_filtered = train_df[train_df['movieid'].isin(top_movies)]

training, data_index = transform_indices(train_filtered, 'userid', 'movieid')
testset = reindex_data(test_df, data_index, entities='items')

# Step 4: Define the data description dictionary
data_description = dict(
    users='userid',
    items='movieid',
    feedback='rating',
    n_users=training.userid.nunique(),
    n_items=training.movieid.nunique()
)

_, _, Vt = build_svd_model(training, rank=200, alpha=0.35, decay_rate=1.05)

P_test = fold_in_users(_, Vt, testset, data_description)

scores = svd_scoring(P_test, Vt, topn=20)
#scores = svd_scoring(item_factors, testset, data_description)

# Step 8: Downvote previously seen items to prevent re-recommendation
downvote_seen_items(scores, testset, data_description)

# Step 9: Generate top-20 recommendations for test users
recommendations = topn_recommendations(scores, topn=20)

In [172]:
# top_movies = train_df['movieid'].value_counts().nlargest(7250).index

# train_filtered = train_df[train_df['movieid'].isin(top_movies)]

# training, data_index = transform_indices(train_filtered, 'userid', 'movieid')
# testset = reindex_data(test_df, data_index, entities='items')

# data_description = dict(
#     users='userid',
#     items='movieid',
#     feedback='rating',
#     n_users=training.userid.nunique(),
#     n_items=training.movieid.nunique()
# )

# U, sigma, Vt = build_svd_model(training, rank=250, alpha=0.35, decay_rate = 0.93)
# item_factors = Vt.T

# scores = svd_scoring(item_factors, testset, data_description)
# downvote_seen_items(scores, testset, data_description)

# recommendations = topn_recommendations(scores, topn=20)

In [173]:
reverse_user_id_map = {new: old for new, old in zip(range(len(data_index['users'])), data_index['users'])}
reverse_movie_id_map = {new: old for new, old in zip(range(len(data_index['items'])), data_index['items'])}

test_users_ids = pd.Index(testset['userid'].drop_duplicates())

recommendations_df = recommendations_to_df(recommendations, test_users_ids, reverse_movie_id_map)

In [166]:
hr, mrr, cov, ndcg = calculate_metrics(recommendations_df, testset, k=20)
print(f"HR@20: {hr:.3}")
print(f"MRR@20: {mrr:.3}")
print(f"Cov@20: {cov:.3}")
print(f"NDCG@20: {ndcg:.3}")

HR@20: 0.886
MRR@20: 0.475
Cov@20: 0.345
NDCG@20: 0.208


In [124]:
hr, mrr, cov, ndcg = calculate_metrics(recommendations_df, testset, k=20)
print(f"HR@20: {hr:.3}")
print(f"MRR@20: {mrr:.3}")
print(f"Cov@20: {cov:.3}")
print(f"NDCG@20: {ndcg:.3}")

HR@20: 0.889
MRR@20: 0.476
Cov@20: 0.334
NDCG@20: 0.211


Hit Rate is quite good: we correctly guess 18/20 what to watch, Mean Reciprocical Rate tells us that on average we recommend good item on the 2nd position - good enough too! Coverage is ok - it shows us that our recommendation contain almost 1/3 of all different movies. NDCG helps consider both the relevance of items and their positions in the list

In [111]:
recommendations_df.head(21)

,userid,movieid
0,55,194
1,55,222
2,55,755
3,55,750
4,55,110
5,55,1586
6,55,1411
7,55,103
8,55,120
9,55,213


In [95]:
recommendations_df.head(21)

,userid,movieid
0,55,194
1,55,222
2,55,755
3,55,750
4,55,1411
5,55,110
6,55,1528
7,55,1586
8,55,213
9,55,120


## Final submit

In [126]:
recommendations_df.to_csv('submission.csv', index=False)